# Initialize

## Import packages

In [17]:
%matplotlib inline

import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull, distance_matrix
import seaborn as sns
import json
from datetime import date, datetime
import os
import matplotlib.animation as animation
from IPython.display import HTML, clear_output, display
import matplotlib
import re
#matplotlib.use('Agg')  # Set the backend to 'agg'


import pandas as pd
from Bio.PDB import PDBParser

# Global variables
XYZ = ['x', 'y', 'z']

## Declare functions

In [2]:
## Treat sets of points as pd.DataFrame
## Treat individial points as pd.Series    

def read_pdb(pdb_file):
    '''Take a PDB file and convert it to a pandas dataframe'''
    parser = PDBParser(PERMISSIVE=1)
    structure = parser.get_structure("structure", pdb_file)
    data = []
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    data.append({"modelNo": model.id,
                                 "chain": chain.id,
                                 "residue": residue.get_id()[1],
                                 "amino_acid": residue.get_resname(),
                                 "atom_name": atom.get_name(),
                                 "x": atom.get_coord()[0],
                                 "y": atom.get_coord()[1],
                                 "z": atom.get_coord()[2],
                                 })
    df = pd.DataFrame(data)
    return df

def get_distance_matrix(df, only_CA=True):
    """Called inside of his_brace_check"""
    # Make sure df only has alpha carbons
    if only_CA and "atom_name" in df:
        df = df[df.atom_name == "CA"]
    # Or keep only one atom per residue
    if only_CA and 'residue' in df:
        df = df.drop_duplicates('residue')
        df = df.set_index('residue')
    elif not only_CA and 'residue' in df:
        df = df.set_index(['residue', 'atom_name'])
    else:
        df = df.reset_index(drop=True)
    # calculate the distance matrix with the function from scipy.spatial
    dists_his = pd.DataFrame(distance_matrix(df[['x','y','z']].values, df[['x','y','z']].values),
                         index=df.index, columns=df.index)
    # Reduce to the minimum distance between the residues
    try:
        dists_his = dists_his.groupby('residue').min().groupby('residue', axis=1).min()
    except Exception as e:
        print("get_distance_matrix error: {}".format(e))
    return dists_his

def get_MMH(df, n_his, his_brace_thres=10, only_CA=True):
    '''Check if the N-terminal histidine is close to another histidine'''
    # Get df with only HIS residues
    df_his = df[df.amino_acid == "HIS"]
    # Get distances
    dists_his = get_distance_matrix(df_his, only_CA=only_CA)
    # Check if there is any other His within the threshold from n_his
    if (dists_his[n_his] <= his_brace_thres).drop(n_his).any():
        is_his_brace = True
    else:
        is_his_brace = False
    return is_his_brace
    

In [9]:
df_dbcan = pd.read_csv('../04-find_motifs/04-dbcan_result.tsv', sep='\t', header=0)
df_dbcan.head()

,Gene ID,EC#,HMMER,dbCAN_sub,DIAMOND,Signalp,#ofTools
0,sp|A3KKC4|LCEMO_STRA7,1.14.99.54|1.14.99.53,AA10(35-225)+CBM2(265-353),AA10_e30+CBM2_e70,AA10+CBM2,N,3
1,tr|A0A014N4U1|A0A014N4U1_9ACTN,1.14.99.54|1.14.99.53,AA10(48-239)+CBM2(274-362),AA10_e30+CBM2_e70,AA10+CBM2,N,3
2,tr|A0A060ZR13|A0A060ZR13_9ACTN,1.14.99.54|1.14.99.53,AA10(48-239)+CBM2(277-367),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-48),3
3,tr|A0A066U3Z1|A0A066U3Z1_9PSEU,1.14.99.54|1.14.99.53,AA10(32-220)+CBM2(273-367),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-32),3
4,tr|A0A066YW87|A0A066YW87_9ACTN,1.14.99.54|1.14.99.53,AA10(33-223)+CBM2(273-363),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-33),3


In [50]:
re.match("AA10\((.*)\)", "AA10(35-225)+CBM2(265-353)",)[1]

'35-225)+CBM2(265-353'

In [14]:
#Extract the accession number
df_dbcan['ID'] = df_dbcan['Gene ID'].apply(lambda x: x.split('|')[1])
# Extract the start and end of AA10
def get_AA10(s):
    if "AA10" in s:
        re.match()
# Extract the start and end of AA10


In [15]:
df_dbcan.head()

,Gene ID,EC#,HMMER,dbCAN_sub,DIAMOND,Signalp,#ofTools,ID
0,sp|A3KKC4|LCEMO_STRA7,1.14.99.54|1.14.99.53,AA10(35-225)+CBM2(265-353),AA10_e30+CBM2_e70,AA10+CBM2,N,3,A3KKC4
1,tr|A0A014N4U1|A0A014N4U1_9ACTN,1.14.99.54|1.14.99.53,AA10(48-239)+CBM2(274-362),AA10_e30+CBM2_e70,AA10+CBM2,N,3,A0A014N4U1
2,tr|A0A060ZR13|A0A060ZR13_9ACTN,1.14.99.54|1.14.99.53,AA10(48-239)+CBM2(277-367),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-48),3,A0A060ZR13
3,tr|A0A066U3Z1|A0A066U3Z1_9PSEU,1.14.99.54|1.14.99.53,AA10(32-220)+CBM2(273-367),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-32),3,A0A066U3Z1
4,tr|A0A066YW87|A0A066YW87_9ACTN,1.14.99.54|1.14.99.53,AA10(33-223)+CBM2(273-363),AA10_e30+CBM2_e70,AA10+CBM2,Y(1-33),3,A0A066YW87


In [ ]:
%%time
import warnings
from tqdm import tqdm

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    #Run all
    failedpdbs = {}
    errorsfile = f'errors_lpmo_sbs_{datetime.now().isoformat(sep="T", timespec="seconds").replace(":",".")}.txt'
    for up in tqdm(allups):
        try:
            pdbfile = './LPMO/all_AF_models/AF-{}-F1-model_v4.pdb'.format(up)
            paefile = './LPMO/all_AF_models/AF-{}-F1-predicted_aligned_error_v4.json'.format(up)
            process_pdb(pdbfile, paefile, plot=False, write=True, printresult=False)
        except Exception as e:
            with open(errorsfile, 'a') as f:
                f.write("Error in {}: {}\n".format(up, e))
            failedpdbs[up] = e

### Read the results file

In [ ]:
# Read the results file
df_hphob = pd.read_csv('./2023-06-16-results_hphob.csv', sep=';', header=0)
df_hphob.head()

In [ ]:
df_hphob.is_his_brace.value_counts()

In [ ]:
# Filter out false
df_hphob_filt = df_hphob[df_hphob.is_his_brace]

In [ ]:
threshold = 140
plt.figure(figsize=(10,6))
ax = sns.histplot(df_hphob_filt.cat_dom_length, color='g')
plt.title("Length of catalytic domain")
ax.axvline(threshold, linestyle=":", color="r", linewidth=2);

In [ ]:
# Filter those with shorter cat_dom
df_hphob_filt = df_hphob_filt[df_hphob_filt.cat_dom_length > threshold]

In [ ]:
# Check the hidrophobic score distribution
plt.figure(figsize=(14,5))
plt.subplot(2,1,1)
ax = sns.histplot(df_hphob.hphob_fraction)
ax.axvline(0.3, linestyle=":", color="r", linewidth=2)
ax.set_xlabel('')
plt.yscale('log')
plt.subplot(2,1,2, sharex=ax)
ax2 = sns.histplot(df_hphob_filt.hphob_fraction)
ax2.axvline(0.3, linestyle=":", color="r", linewidth=2)
plt.yscale('log')

In [ ]:
# Get those with > 0.3 hydrophobicity
df_hphob_filt = df_hphob_filt[df_hphob_filt.hphob_fraction >= 0.3].sort_values('hphob_fraction', ascending=False)
df_hphob_filt.head(20)

In [ ]:
# save the filtered dataframe
df_hphob_filt.drop_duplicates().to_csv('results_hphob/2023-06-16-filtered_results.csv', sep=';', index=False)

In [ ]:
# Read results
df_hphob_filt = pd.read_csv('results_hphob/2023-06-16-filtered_results.csv', sep=';', header=0)

In [ ]:
# Checking an individual LPMO
#up = 'A0A2L2BFS1'
#pdbfile = './LPMO/all_AF_models/AF-{}-F1-model_v4.pdb'.format(up)
#paefile = './LPMO/all_AF_models/AF-{}-F1-predicted_aligned_error_v4.json'.format(up)

#res = process_pdb(pdbfile, paefile, plot=True, write=False, printresult=True, return_dict=True, animate=False)

### MSA

In [ ]:
# Get the 
from Bio import SeqIO
# Get the fasta file downloaded from UniProt using the IDs (https://www.uniprot.org/id-mapping/)
fastafile = './uniprot-download_true_format_fasta-2023.06.20-06.41.17.90.fasta'
recs = list(SeqIO.parse(fastafile, 'fasta'))

for r in recs:
    # Make the id only the uniprot ID
    r.id = r.id.split('|')[1]
    # Get the start and end of the CD
    s = df_hphob_filt.set_index('uniprot').loc[r.id]
    # Get only the catalytic domain
    r.seq = r.seq[int(s.cat_dom_start)-1:int(s.cat_dom_end)]

cd_fasta = 'top_hphob_cd.fasta'
with open(cd_fasta, 'w') as f:
    print(f'Writing fasta file of catalytic domains: {cd_fasta}')
    SeqIO.write(recs, f, 'fasta')

### Cluster and select representatives
Cluster using cd-hit (https://anaconda.org/bioconda/cd-hit, run in command line)
```
mkdir cd_hit
cd cd_hit
cd-hit -i ../top_hphob_cd.fasta -c 0.7 -o reduced_cdhit70.fa
```


In [ ]:
with open('cd_hit/reduced_cdhit70.fa.clstr', 'r') as f:
    clst_raw = f.read()

In [ ]:
clst = clst_raw.split('>Cluster')
clst = [i.split('\n') for i in clst if i != '']
clst_dict = dict([(i[0], i[1:-1]) for i in clst])

In [ ]:
for k in clst_dict:
    clst_dict[k] = [v.split('>')[1].split('.')[0] for v in clst_dict[k]]

In [ ]:
clst_inv_dict = {}
for k, v in clst_dict.items():
    for i in v:
        clst_inv_dict[i] = int(k)

In [ ]:
df_hphob_filt['clust70'] = df_hphob_filt.uniprot.apply(lambda x: clst_inv_dict[x])

In [ ]:
df_hphob_filt.set_index('uniprot', inplace=True)

In [ ]:
df_cazy_lpmos = pd.read_csv('./LPMO/CAZy_LPMOs_uniprot.csv')
df_cazy_lpmos['uniprot'] = df_cazy_lpmos.uniprot.apply(lambda x: str(x).replace(' ', ''))
df_cazy_lpmos.set_index('uniprot', inplace=True)

In [ ]:
df_hphob_filt = (
    df_hphob_filt
    .join(df_cazy_lpmos, how='left')
    .sort_values(by='clust70')
    .sort_values(by='hphob_fraction', ascending=False)
)

In [ ]:
df_hphob_filt.sort_values(by='clust70')

In [ ]:
df_hphob_filt.reset_index().to_excel(f'results_hphob/{date.today()}-filtered_clust70_info.xlsx', index=False)

In [ ]:
# keep only one representative of each cluster (the one with highest hphob)
df_top_per_clust = df_hphob_filt.drop_duplicates('clust70')

### Make a script to visualize in PyMol

In [ ]:
# Function for each LPMO
def ind_lpmo_script(s, script=f''):
    # Import structure
    if 'uniprot' in s:
        upid = s.uniprot
    else:
        upid = s.name
    script += f'\n# Adding the structure {upid}\n'
    script += 'disable all\n'
    script += f'fetchAF2 {upid}, path="./fake/path/"\n'
    # Change the color of the whole structure
    script += f'color blue, {upid}\n'

    # Format catalytic domain
    script += f'select {upid} and resi {s.cat_dom_start}-{s.cat_dom_end}\n'
    script += f'color orange, sele\n'

    # Format the surface residues
    surflist = '+'.join(s.close_residues.split(','))
    script += f'select {upid} and resi {surflist}\n'
    script += f'color green, sele\n'
    script += f'show sticks, sele\n'
    
    # Format the N-His residues
    script += f'select {upid} and resi {s.N_his}\n'
    script += f'color red, sele\n'
    script += f'show sticks, sele\n'
    
    return script


def make_script(df, session=f'Top{n}_LPMOs_{date.today()}.pse', scriptname=f'script_viz_top{n}_{date.today()}.pml'):
    
    # Write script header
    script = '''
# Visualize most hydrophobic LPMOS

# When fetching, a fake path is used to avoid writing a file for each structure.
'''

    
    for i in df.index:
        script = ind_lpmo_script(df.loc[i], script)

    # Add the last part of the script
    script += f'''
# Final formatting
color atomic, not elem C
bg_color grey90

# Save PyMOL session
save {session}

## End of script
'''

    # write the script to a file
    if not os.path.isdir('pymol'):
        os.mkdir('pymol')
    with open('pymol/'+scriptname, 'w') as f:
        f.writelines(script)

# Run the script generator for the top n LPMOs
n = 15
make_script(df_hphob_filt.head(n))

In [ ]:
print(ind_lpmo_script(df_hphob_filt.loc['A0A2T5P0S3']))

In [ ]:
scriptname = f'top_per_clust_{date.today()}'
make_script(df_top_per_clust, session=scriptname+'.pse', scriptname=scriptname+'.pml')